In [1]:
!pip install psutil requests pyttsx3 SpeechRecognition pyaudio


  Using cached PyAudio-0.2.14-cp312-cp312-win_amd64.whl.metadata (2.7 kB)
   ---------------------------------------- 0.0/32.9 MB ? eta -:--:--
    --------------------------------------- 0.5/32.9 MB 4.2 MB/s eta 0:00:08
   - -------------------------------------- 1.6/32.9 MB 3.8 MB/s eta 0:00:09
   -- ------------------------------------- 2.4/32.9 MB 4.1 MB/s eta 0:00:08
   ---- ----------------------------------- 3.4/32.9 MB 4.2 MB/s eta 0:00:08
   ----- ---------------------------------- 4.7/32.9 MB 4.5 MB/s eta 0:00:07
   ------ --------------------------------- 5.5/32.9 MB 4.4 MB/s eta 0:00:07
   ------- -------------------------------- 6.6/32.9 MB 4.5 MB/s eta 0:00:06
   --------- ------------------------------ 7.9/32.9 MB 4.8 MB/s eta 0:00:06
   ----------- ---------------------------- 9.2/32.9 MB 4.9 MB/s eta 0:00:05
   ------------- -------------------------- 10.7/32.9 MB 5.2 MB/s eta 0:00:05
   -------------- ------------------------- 12.3/32.9 MB 5.4 MB/s eta 0:00:04
   ----

In [2]:
!pip install pipwin
!pipwin install pyaudio


  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   -------

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\P R VISHAL\anaconda3\Scripts\pipwin.exe\__main__.py", line 2, in <module>
  File "c:\Users\P R VISHAL\anaconda3\Lib\site-packages\pipwin\command.py", line 28, in <module>
    from . import pipwin, __version__
  File "c:\Users\P R VISHAL\anaconda3\Lib\site-packages\pipwin\pipwin.py", line 13, in <module>
    import js2py
  File "c:\Users\P R VISHAL\anaconda3\Lib\site-packages\js2py\__init__.py", line 72, in <module>
    from .base import PyJsException
  File "c:\Users\P R VISHAL\anaconda3\Lib\site-packages\js2py\base.py", line 2965, in <module>
    @Js
     ^^
  File "c:\Users\P R VISHAL\anaconda3\Lib\site-packages\js2py\base.py", line 165, in Js
    return PyJsFunction(val, FunctionPrototype)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\P R VISHAL\anaconda3\Lib\site-packages\js2py\base.py", line 1377, in __init_

In [3]:
import os
import json
import subprocess
import threading
import webbrowser
import time
import urllib.parse
import psutil
import requests
import tkinter as tk
from tkinter import filedialog, scrolledtext, messagebox
from datetime import datetime

# Optional imports with graceful fallback
try:
    import pyttsx3
except ImportError:
    pyttsx3 = None

try:
    import speech_recognition as sr
except ImportError:
    sr = None

try:
    import pvporcupine
    from pvrecorder import PvRecorder
except ImportError:
    pvporcupine = None
    PvRecorder = None

try:
    import ollama
except ImportError:
    ollama = None

try:
    import PyPDF2
except ImportError:
    PyPDF2 = None

try:
    from docx import Document
except ImportError:
    Document = None

try:
    from bs4 import BeautifulSoup
except ImportError:
    BeautifulSoup = None

try:
    from duckduckgo_search import DDGS
except ImportError:
    DDGS = None

# ==================================================
# CONFIGURATION
# ==================================================
MEMORY_FILE = "friday_memory.json"
NOTES_FILE = "friday_notes.txt"
PASSWORD = "1234"  # Change this to your desired password
ACCESS_KEY = ""    # Picovoice Access Key (optional)
WAKE_WORD_PATH = ""  # Path to custom wake word .ppn file (optional)

# ==================================================
# VOICE ENGINE
# ==================================================
class VoiceEngine:
    def __init__(self):
        self.enabled = pyttsx3 is not None
        self.lock = threading.Lock()
        if self.enabled:
            self.engine = pyttsx3.init(driverName="sapi5")
            self.engine.setProperty("rate", 165)
            self.engine.setProperty("volume", 1.0)
            self.engine.say(" ")
            self.engine.runAndWait()

    def speak(self, text):
        print("FRIDAY:", text)
        if not self.enabled:
            return
        with self.lock:
            try:
                self.engine.stop()
                self.engine.say(text)
                self.engine.runAndWait()
            except:
                pass

    def listen_once(self):
        if sr is None:
            print("Speech recognition not available.")
            return ""
        r = sr.Recognizer()
        with sr.Microphone() as source:
            self.speak("Listening...")
            try:
                audio = r.listen(source, timeout=5, phrase_time_limit=10)
            except sr.WaitTimeoutError:
                return ""
        try:
            text = r.recognize_google(audio)
            print("You (voice):", text)
            return text.lower()
        except:
            self.speak("I could not understand.")
            return ""

    def continuous_listen(self):
        if pvporcupine and ACCESS_KEY and WAKE_WORD_PATH:
            try:
                porcupine = pvporcupine.create(
                    access_key=ACCESS_KEY,
                    keyword_paths=[WAKE_WORD_PATH]
                )
                recorder = PvRecorder(frame_length=porcupine.frame_length)
                recorder.start()
                self.speak("Continuous listening activated. Say 'Hey Friday'")
                while True:
                    pcm = recorder.read()
                    if porcupine.process(pcm) >= 0:
                        self.speak("Yes sir?")
                        cmd = self.listen_once()
                        recorder.stop()
                        porcupine.delete()
                        recorder.delete()
                        return cmd or ""
            except Exception as e:
                print("Wake word error:", e)

        self.speak("Wake word not set. Using manual voice mode.")
        return self.listen_once()


# ==================================================
# MEMORY ENGINE
# ==================================================
class MemoryEngine:
    def __init__(self):
        default = {
            "short": [],
            "long": [],
            "context": {
                "last_action": "",
                "last_topic": "",
                "last_search": ""
            }
        }
        if not os.path.exists(MEMORY_FILE):
            with open(MEMORY_FILE, "w") as f:
                json.dump(default, f, indent=2)
        self.load()
        if "context" not in self.data:
            self.data["context"] = default["context"]
            self.save()

    def load(self):
        with open(MEMORY_FILE, "r") as f:
            self.data = json.load(f)

    def save(self):
        with open(MEMORY_FILE, "w") as f:
            json.dump(self.data, f, indent=2)

    def update_context(self, key, value):
        self.data["context"][key] = value
        self.save()

    def get_context(self, key):
        return self.data["context"].get(key, "")

    def get_recent_commands(self, n=5):
        commands = []
        for i in range(len(self.data["short"]) - 1, -1, -2):  # Step backwards, user messages
            if self.data["short"][i]["role"] == "user":
                commands.append(self.data["short"][i]["content"])
            if len(commands) >= n:
                break
        return commands[::-1]  # Reverse to chronological order

    def store_conversation(self, user, assistant):
        self.data["short"].append({"role": "user", "content": user})
        self.data["short"].append({"role": "assistant", "content": assistant})
        if len(self.data["short"]) > 30:
            self.data["long"].extend(self.data["short"][:10])
            self.data["short"] = self.data["short"][10:]
        self.save()


# ==================================================
# INTENT ENGINE
# ==================================================
class IntentEngine:
    def analyze(self, text, history=[], context={}):
        text = text.lower().strip()

        if "open youtube" in text:
            return "OPEN_YOUTUBE", {}
        if "play" in text and "youtube" in text:
            query = text.replace("play", "").replace("on youtube", "").strip()
            return "PLAY_YOUTUBE", {"query": query}
        if "play next" in text or "next video" in text:
            return "PLAY_NEXT", {}
        if text.startswith("open "):
            app = text[5:].strip()
            return "OPEN_APP", {"app_name": app}
        if any(phrase in text for phrase in ["tell me about", "who is", "what is", "explain"]):
            topic = text.split(maxsplit=3)[-1] if len(text.split()) > 3 else text
            return "KNOWLEDGE", {"topic": topic}
        if "research" in text:
            topic = text.replace("research", "").strip()
            return "RESEARCH", {"topic": topic}
        if any(word in text for word in ["battery"]):
            return "BATTERY_STATUS", {}
        if any(word in text for word in ["ram", "memory usage", "cpu"]):
            return "SYSTEM_STATUS", {}
        if any(phrase in text for phrase in ["memory status", "recent commands", "what did i say", "do you remember"]):
            return "MEMORY_STATUS", {}
        if "autonomous on" in text:
            return "AUTONOMOUS_ON", {}
        if "autonomous off" in text:
            return "AUTONOMOUS_OFF", {}
        if "exit" in text or "quit" in text or "stop" in text:
            return "EXIT", {}

        return "GENERAL", {}


# ==================================================
# TOOLS
# ==================================================
class Tools:
    APP_PATHS = {
        "chrome": r"C:\Program Files\Google\Chrome\Application\chrome.exe",
        "notepad": "notepad.exe",
        "calculator": "calc.exe",
        "edge": "msedge",
        "word": r"C:\Program Files\Microsoft Office\root\Office16\WINWORD.EXE",
        "excel": r"C:\Program Files\Microsoft Office\root\Office16\EXCEL.EXE",
        "powerpoint": r"C:\Program Files\Microsoft Office\root\Office16\POWERPNT.EXE",
        "explorer": "explorer.exe"
    }

    def open_app(self, name):
        name = name.lower()
        if name == "whatsapp":
            webbrowser.open("https://web.whatsapp.com")
            return "WhatsApp opened"
        if name in self.APP_PATHS:
            subprocess.Popen(self.APP_PATHS[name], shell=True)
            return f"{name.capitalize()} opened"
        return "App not supported"

    def open_youtube(self):
        webbrowser.open("https://www.youtube.com")
        return "YouTube opened"

    def play_youtube(self, query):
        url = f"https://www.youtube.com/results?search_query={urllib.parse.quote(query)}"
        webbrowser.open(url)
        return f"Searching YouTube for '{query}'"

    def play_next(self, last_topic):
        if not last_topic:
            return "No previous topic to continue from."
        return self.play_youtube(last_topic + " next episode")

    def detailed_battery_status(self):
        battery = psutil.sensors_battery()
        if not battery:
            return "No battery detected. This is likely a desktop."

        percent = battery.percent
        power_plugged = battery.power_plugged
        secs_left = battery.secsleft

        status = "Charging" if power_plugged else "Running on battery"
        if secs_left == psutil.POWER_TIME_UNLIMITED:
            time_left = "Fully charged" if power_plugged else "Calculating..."
        elif secs_left == psutil.POWER_TIME_UNKNOWN:
            time_left = "Unknown time remaining"
        else:
            hours = secs_left // 3600
            mins = (secs_left % 3600) // 60
            time_left = f"Approximately {hours} hours {mins} minutes remaining"

        return f"Battery level is {percent}%. Laptop is {status.lower()}. {time_left}."

    def system_status(self):
        cpu = psutil.cpu_percent(interval=1)
        ram = psutil.virtual_memory()
        ram_percent = ram.percent
        ram_used = round(ram.used / (1024**3), 2)
        ram_total = round(ram.total / (1024**3), 2)
        return f"CPU usage: {cpu}%. RAM usage: {ram_percent}% ({ram_used} GB used out of {ram_total} GB)."

    def knowledge(self, topic, voice):
        try:
            url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{topic.replace(' ', '_')}"
            res = requests.get(url, timeout=8).json()
            extract = res.get("extract", "No information found.")
            lines = extract.split(". ")[:5]
            summary = ". ".join(lines) + "."
            voice.speak(summary)
            voice.speak("Do you want me to open the full article?")
            reply = voice.listen_once()
            if "yes" in reply or "open" in reply:
                webbrowser.open(f"https://en.wikipedia.org/wiki/{topic.replace(' ', '_')}")
                return summary + " Opening full page."
            return summary
        except:
            webbrowser.open(f"https://www.google.com/search?q={topic}")
            return "Searching the web for " + topic

    def research(self, topic, voice):
        webbrowser.open(f"https://www.google.com/search?q={topic}")
        return f"Opened search results for '{topic}' in browser."


# ==================================================
# EXECUTOR
# ==================================================
class Executor:
    def __init__(self):
        self.tools = Tools()

    def execute(self, intent, params, memory, voice):
        if intent == "OPEN_APP":
            return self.tools.open_app(params.get("app_name", ""))
        if intent == "OPEN_YOUTUBE":
            return self.tools.open_youtube()
        if intent == "PLAY_YOUTUBE":
            query = params.get("query", "")
            memory.update_context("last_topic", query)
            return self.tools.play_youtube(query)
        if intent == "PLAY_NEXT":
            last = memory.get_context("last_topic")
            return self.tools.play_next(last)
        if intent == "KNOWLEDGE":
            return self.tools.knowledge(params.get("topic", ""), voice)
        if intent == "BATTERY_STATUS":
            return self.tools.detailed_battery_status()
        if intent == "SYSTEM_STATUS":
            return self.tools.system_status()
        if intent == "MEMORY_STATUS":
            recent = memory.get_recent_commands(6)
            last_action = memory.get_context("last_action")
            if recent:
                response = "Here are your recent commands:\n" + "\n".join(f"- {cmd}" for cmd in recent)
                if last_action:
                    response += f"\n\nYour last action was: {last_action}"
                return response
            return "No recent commands in memory."
        if intent == "RESEARCH":
            return self.tools.research(params.get("topic", ""), voice)
        return "I didn't understand that command."


# ==================================================
# GUI DASHBOARD
# ==================================================
class Dashboard:
    def __init__(self, friday):
        self.friday = friday
        self.root = tk.Tk()
        self.root.title("FRIDAY - Agentic Assistant")
        self.root.geometry("900x650")
        self.root.configure(bg="#1e1e1e")

        tk.Label(self.root, text="FRIDAY", font=("Arial", 20, "bold"), fg="#00ffcc", bg="#1e1e1e").pack(pady=15)

        self.status = tk.Label(self.root, text="Status: Online", fg="lime", bg="#1e1e1e", font=("Arial", 12))
        self.status.pack()

        self.logs = scrolledtext.ScrolledText(self.root, height=22, bg="#2d2d2d", fg="white", font=("Consolas", 10))
        self.logs.pack(padx=15, pady=10, fill=tk.BOTH, expand=True)

        btn_frame = tk.Frame(self.root, bg="#1e1e1e")
        btn_frame.pack(pady=10)
        tk.Button(btn_frame, text="Upload Document", command=self.upload, bg="#333", fg="white", width=15).grid(row=0, column=0, padx=15)
        tk.Button(btn_frame, text="View Memory", command=self.view_memory, bg="#333", fg="white", width=15).grid(row=0, column=1, padx=15)

        threading.Thread(target=self.root.mainloop, daemon=True).start()

    def log(self, text):
        self.logs.insert(tk.END, f"{datetime.now().strftime('%H:%M:%S')} | {text}\n")
        self.logs.see(tk.END)

    def upload(self):
        path = filedialog.askopenfilename(filetypes=[("Documents", "*.pdf *.docx *.txt")])
        if path:
            self.friday.process_document(path)

    def view_memory(self):
        try:
            with open(MEMORY_FILE, "r") as f:
                content = f.read()
            messagebox.showinfo("Memory", content[:4000] + ("\n..." if len(content) > 4000 else ""))
        except Exception as e:
            messagebox.showerror("Error", str(e))


# ==================================================
# FRIDAY CORE
# ==================================================
class FRIDAY:
    def __init__(self):
        self.voice = VoiceEngine()
        self.memory = MemoryEngine()
        self.intent_engine = IntentEngine()
        self.executor = Executor()
        self.dashboard = Dashboard(self)
        self.autonomous = False

    def authenticate(self):
        self.voice.speak("Please say your password.")
        for _ in range(3):
            said = self.voice.listen_once()
            if said == PASSWORD.lower():
                self.voice.speak("Authentication successful. FRIDAY online, sir.")
                return True
            self.voice.speak("Incorrect password.")
        self.voice.speak("Access denied.")
        return False

    def process_document(self, path):
        self.dashboard.log(f"Document uploaded: {os.path.basename(path)}")
        self.voice.speak("Document received. Ready for analysis if needed.")

    def run(self):
        if not self.authenticate():
            return

        self.dashboard.log("System initialized")
        self.voice.speak("Ready for your command, sir.")

        while True:
            mode = input("\nPress 'v' for Voice, 't' for Text, or 'q' to quit: ").lower().strip()

            if mode == 'q':
                self.voice.speak("Goodbye, sir.")
                break
            elif mode == 'v':
                command = self.voice.continuous_listen()
            elif mode == 't':
                command = input("You (text): ").strip()
            else:
                print("Please type 'v', 't', or 'q'.")
                continue

            if not command:
                continue

            self.dashboard.log(f"You: {command}")

            history = self.memory.data["short"]
            context = self.memory.data["context"]
            intent, params = self.intent_engine.analyze(command, history, context)

            if intent == "EXIT":
                self.voice.speak("Shutting down. Have a great day.")
                break

            result = self.executor.execute(intent, params, self.memory, self.voice)

            # Update last action
            self.memory.update_context("last_action", command)

            self.memory.store_conversation(command, result)
            self.dashboard.log(f"FRIDAY: {result}")
            self.voice.speak(result)


# ==================================================
# RUN
# ==================================================
if __name__ == "__main__":
    FRIDAY().run()

Exception in thread Thread-11 (mainloop):
Traceback (most recent call last):
  File "c:\Users\P R VISHAL\anaconda3\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "c:\Users\P R VISHAL\anaconda3\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\P R VISHAL\anaconda3\Lib\tkinter\__init__.py", line 1505, in mainloop
    self.tk.mainloop(n)
RuntimeError: Calling Tcl from different apartment


FRIDAY: Please say your password.
FRIDAY: Listening...
You (voice): 1234
FRIDAY: Authentication successful. FRIDAY online, sir.
FRIDAY: Ready for your command, sir.
FRIDAY: YouTube opened
Please type 'v', 't', or 'q'.
FRIDAY: App not supported
FRIDAY: App not supported
FRIDAY: I didn't understand that command.
FRIDAY: Searching YouTube for 'ro in youtube'
FRIDAY: Opened search results for 'rohit' in browser.
FRIDAY: No recent commands in memory.
FRIDAY: No recent commands in memory.
FRIDAY: No recent commands in memory.
FRIDAY: CPU usage: 1.8%. RAM usage: 82.0% (12.87 GB used out of 15.69 GB).
FRIDAY: Goodbye, sir.
